In [2]:
import pandas as pd
import numpy as np
import random as rng
from schedule import Schedule
from paintshop import PaintShop

# Settings

In [2]:
SEED = 420

# Setup

In [ ]:
rng.seed(SEED)
PS = PaintShop()

# Classes

Metaheuristics:
- Multistart:  Improving multiple starts and take the best local optimum
- Taboo Search: Reverse steps are forbidden (taboo-list). Tabu moves are removed from the list after a number of moves. Stop at a certain number of iterations OR when all available moves are taboo. Keep track of incumbent solution.
- Simulated Annealing: Temperature & cooling schedule. Allways accepts improving moves. Non-improving moves with probablity based on the obj. improvement and temperature. Randomly choose a move, compute the gain. If it improves, accept, otherwise accept with probability e^(delta_obj / q). Update incumbent solution. Reduce temperature q. See graph in slides.
- Genetic algorithms: Many different ideas. Population of solutions.
 - Start with a number of random solutions.
 - Create new solutions by combining pairs.
 - Mutations sometimes.
 - Select survivors (elite).



# Solution

In [40]:
def heuristic_constructive_simple() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # For each order (ordered by order-number ascending).
    for order_id in PS.order_ids:
        
        # Determine machine index with the shortest queue (adding a machine's index (scaled to a fraction) works like the tiebreaking rule).
        machine_id_next = sorted(
            PS.machine_ids, 
            key = lambda i:
                len(schedule[i, :]) +
                i / len(PS.machine_ids)
        )[0]
        
        # Add order to machine queue.
        schedule[machine_id_next, :] += [order_id]
        
    return schedule

In [45]:
def heuristic_constructive_simple_2() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # For each order (ordered by order-number ascending).
    for order_id in PS.order_ids:
        
        # Determine machine index with the shortest queue (adding a machine's index (scaled to a fraction) works like the tiebreaking rule).
        machine_id_next = sorted(
            PS.machine_ids, 
            key = lambda i:
                schedule.get_finish_time(i) + 
                i / len(PS.machine_ids), 
        )[0]
        
        # Add order to machine queue.
        schedule[machine_id_next] += [order_id]
        
    return schedule

In [46]:
def heuristic_constructive_simple_3() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # For each order (ordered by order-number ascending).
    for order_id in sorted(PS.order_ids, key = lambda order_id: PS.orders.loc[order_id, 'deadline']):
        
        # Determine machine index with the shortest queue (adding a machine's index (scaled to a fraction) works like the tiebreaking rule).
        machine_id_next = sorted(
            PS.machine_ids, 
            key = lambda i:
                schedule.get_finish_time(i) + 
                i / len(PS.machine_ids), 
        )[0]
        
        # Add order to machine queue.
        schedule[machine_id_next] += [order_id]
        
    return schedule

In [47]:
def heuristic_constructive_random() -> Schedule:
    """Constructs a solution according to the following heuristic:
    1. Create an empty solution. (Empty lists (representing order-numbers) by machine-numbers in a dictionary)
    2. Assign the order with the lowest order-number to the machine with the lowest amount of assigned orders, adding it to the end to the order-queue for that machine. The tiebreaking rule is that the machine with the lowest machine-number gets the order.
    3. Go to step 2 unless all orders are assigned.
    
    Returns:
        dict[int, list[int]]: The constructed solution. 
    """
    
    
    # Construct an empty solution dictionary.
    schedule = Schedule()
    
    # Create list of shuffled order ID's
    order_ids_remaining = PS.order_ids
    rng.shuffle(order_ids_remaining)
    
    while len(order_ids_remaining) > 0:
        
        next_order_id_index = rng.choice(range(len(order_ids_remaining)))
        
        schedule[rng.choice(PS.machine_ids)] += [order_ids_remaining[next_order_id_index]]
        
        order_ids_remaining = np.delete(order_ids_remaining, next_order_id_index)      
    return schedule

In [48]:
import math


def heuristic_improvement_best(initial: Schedule) -> Schedule:
    
    current_schedule = initial
    
    print(f"Initial:")
    print(str(current_schedule) + "\n")
    
    while True:
    
        best_swap = None
        best_swap_cost = initial.get_cost()
        
        for swap in current_schedule.get_swaps():
            swapped_schedule: Schedule = current_schedule.get_copy(swap)
            swapped_cost = swapped_schedule.get_cost()
            
            # print(f"Swap: {swap}, Cost: {swapped_cost}")
            
            if (swapped_cost < best_swap_cost):
                best_swap = swap
                best_swap_cost = swapped_cost
        
        # Break the loop if no improving swap was found.
        if best_swap == None:
            break
        
        current_schedule.swap(best_swap)
        print(f"Swapped: {best_swap}.")
        print(str(current_schedule) + "\n")
        
    return current_schedule

In [49]:
def heuristic_improvement_first(initial: Schedule) -> Schedule:
    
    current_schedule = initial
    
    print(f"Initial:")
    print(str(current_schedule) + "\n")
    
    while True:
    
        first_improvement = None
        initial_swap_cost = initial.get_cost()
        
        for swap in current_schedule.get_swaps():
            swapped_schedule: Schedule = current_schedule.get_copy(swap)
            swapped_cost = swapped_schedule.get_cost()
            
            # print(f"Swap: {swap}, Cost: {swapped_cost}")
            # 
            if (swapped_cost < initial_swap_cost):
                first_improvement = swap
                break
        
        # Break the loop if no improving swap was found.
        if first_improvement == None:
            break
        
        current_schedule.swap(first_improvement)
        print(f"Swapped: {first_improvement}.")
        print(str(current_schedule) + "\n")
        
    return current_schedule

In [50]:
# heuristic_improvement_best(heuristic_constructive_simple()).draw()
# heuristic_constructive_simple().draw()

In [ ]:
print('\n\n'.join([
    f"{schedule}" for heuristic, schedule in {
        "constructive_simple": heuristic_constructive_simple(),
        "constructive_simple_2": heuristic_constructive_simple_2(),
        "constructive_simple_3": heuristic_constructive_simple_3(),
        "constructive_random": heuristic_constructive_random()
    }.items()
]))

In [52]:
# import matplotlib.pyplot as plt

# plt.hist([s.get_cost() for s in random_schedules], bins = 30)
# plt.show()

In [53]:
import itertools as iter

sol = heuristic_constructive_simple()

# Gett

In [ ]:
a = heuristic_constructive_simple()
a.get_copy(((0,1), (1,6)))